<a href="https://colab.research.google.com/github/hvmt2003/LLM/blob/main/Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
with open("/content/the-verdict.txt","r",encoding='utf-8') as f:
  raw_text=f.read()

print("Total Number of character:",len(raw_text))
print(raw_text[:99])

Total Number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Our aim is tokenize the entire text.

In [4]:
import re
text="Hello, world. This is a test."
result=re.split(r'(\s)',text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test.']


In [5]:
result1=re.split(r'([,.]|\s)',text)
print(result1)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [6]:
result1=[item for item in result1 if item.strip()]
print(result1)

['Hello', ',', 'world', '.', 'This', 'is', 'a', 'test', '.']


In [7]:
result2=re.split(r'([,.:;?_!"()\']|--|\s)',text)
result2=[item.strip() for item in result if item.strip()]
print(result)

['Hello,', ' ', 'world.', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test.']


In [8]:
preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [9]:
print(len(preprocessed))

4690


Converting the entire token into their respective token ids


In [10]:
all_words=sorted(set(preprocessed))
vocab_size=len(all_words)
print(vocab_size)

1130


In [18]:
vocab={token:integer for integer,token in enumerate(all_words)}

In [19]:
for i,item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [23]:
import re

class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int=vocab
    self.int_to_str={i:s for s,i in vocab.items()}

  def encode(self,text):
    # Updated regex to include double quotes and apostrophes within words
    preprocessed=re.split(r'([,.;:?()"\'!_]|\s|--)',text)

    preprocessed=[
        item.strip() for item in preprocessed if item.strip()
    ]
    ids=[self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    # Improved decoding to handle punctuation
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    text = re.sub(r'\s+(--)\s+', r'\1', text)
    text = re.sub(r"'\s+(\S+)", r"'\1", text) # handles cases like "It 's"
    return text

In [25]:
tokenizer=SimpleTokenizerV1(vocab)
text=""""It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids=tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
" It's the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [26]:
decoded_text = tokenizer.decode(ids)
print(decoded_text)

" It's the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [27]:
# Aksing the tokenizer to encode something not in the dictionary
text="Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

Gpt uses Special Context Tokens, we are going to use this, for the above case

In [31]:
# adding <|unk|> and <|endoftext|>

all_tokens=sorted(list(set(preprocessed)))
all_tokens.extend(["<|unk|>","<|endoftext|>"])
vocab={token:integer for integer,token in enumerate(all_tokens)}

In [32]:
len(vocab.items())

1132

In [33]:
for i, item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|unk|>', 1130)
('<|endoftext|>', 1131)


Version Two of Simple Tokenizer Class

In [34]:
from concurrent.futures import process
import re

class SimpleTokenizerV2:
  def __init__(self,vocab):
    self.str_to_int=vocab
    self.int_to_str={i:s for s,i in vocab.items()}

  def encode(self,text):
    # Updated regex to include double quotes and apostrophes within words
    preprocessed=re.split(r'([,.;:?()"\'!_]|\s|--)',text)

    preprocessed=[
        item.strip() for item in preprocessed if item.strip()
    ]
    preprocessed=[
        item if item in self.str_to_int
        else "<|unk|>" for item in preprocessed
                  ]
    ids=[self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    # Improved decoding to handle punctuation
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    text = re.sub(r'\s+(--)\s+', r'\1', text)
    text = re.sub(r"'\s+(\S+)", r"'\1", text) # handles cases like "It 's"
    return text

In [39]:
# Aksing the tokenizer to encode something not in the dictionary
text1="Hello, do you like tea?"
text2="In the sunlit terraces of the palace."

text="<|endoftext|> ".join((text1,text2))
print(text)

Hello, do you like tea?<|endoftext|> In the sunlit terraces of the palace.


In [40]:
tokenizer=SimpleTokenizerV2(vocab)
tokenizer.encode(text)

[1130, 5, 355, 1126, 628, 975, 10, 1131, 55, 988, 956, 984, 722, 988, 1130, 7]

In [41]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'